In [18]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time 
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Document, Text, Integer
from elasticsearch.helpers import bulk
import numpy as np

In [19]:
df = pd.read_csv('data_youtube.csv',delimiter=',')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1278 entries, 0 to 1277
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Jeu                       1278 non-null   object
 1   Titre                     1278 non-null   object
 2   Chaîne                    1278 non-null   object
 3   Nombre de vues            1278 non-null   int64 
 4   Date de la mise en ligne  1278 non-null   object
 5   Lien de la vidéo          1278 non-null   object
dtypes: int64(1), object(5)
memory usage: 60.0+ KB


In [20]:
data = df.to_dict('records')

In [21]:
data

[{'Jeu': 'League of Legends',
  'Titre': 'The Mageseeker: A League of Legends Story | Official Teaser Trailer',
  'Chaîne': 'Riot Forge',
  'Nombre de vues': 520000,
  'Date de la mise en ligne': 'il y a 4 jours',
  'Lien de la vidéo': 'https://www.youtube.com/watch?v=MsqRA3cRRWg'},
 {'Jeu': 'League of Legends',
  'Titre': '2023 LEC Winter - Playoffs - Round 2 - MAD vs SK',
  'Chaîne': 'LEC',
  'Nombre de vues': 417000,
  'Date de la mise en ligne': 'Diffusé il y a 1 jour',
  'Lien de la vidéo': 'https://www.youtube.com/watch?v=1wsUK891UtI'},
 {'Jeu': 'League of Legends',
  'Titre': 'These 2 are BREAKING League of Legends together...',
  'Chaîne': 'Vandiril',
  'Nombre de vues': 357000,
  'Date de la mise en ligne': 'il y a 6 jours',
  'Lien de la vidéo': 'https://www.youtube.com/watch?v=LrIPVg7H39U'},
 {'Jeu': 'League of Legends',
  'Titre': 'MILIO & NAAFIRI NEW CHAMPIONS TEASERS & RELEASE DATE - League of Legends',
  'Chaîne': 'Remus',
  'Nombre de vues': 245000,
  'Date de la mise e

In [22]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200,"scheme": "http"}])

In [23]:
es.ping()

True

In [24]:
def generate_data(data):
    for docu in data:
        yield {
            "_index": "yt_twitch",
            "_type": "video",
            "_source": {k:v if v else None for k,v in docu.items()},
        }
        
if es.indices.exists('yt_twitch')==True:
    es.indices.delete(index='yt_twitch')
    bulk(es, generate_data(data))
else :
    bulk(es, generate_data(data))

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\2793155111.py:9: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists('yt_twitch')==True:
c:\Users\vgvic\AppData\Local\Programs\Python\Python310\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchWarning)


In [25]:
QUERY={"query": {
    "bool": {
      "must": [],
      "filter": [
        {
          "bool": {
            "should": [
              {
                "match": {
                  "Jeu.keyword": "League of Legends"
                }
              }
            ],
            "minimum_should_match": 1
          }
        }
      ],
      "should": [],
      "must_not": []
    }
  },}

  

In [26]:
result = es.search(index="yt_twitch", body=QUERY, size = 1500)

C:\Users\vgvic\AppData\Local\Temp\ipykernel_6884\3229683340.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  result = es.search(index="yt_twitch", body=QUERY, size = 1500)


In [27]:
[elt['_source'] for elt in result["hits"]["hits"]]

[]